# Optimal Speed and Optimal Hedging

First I propose to solve a deterministic problem with the same parameters. I will help up to:
(1) Compare the results from deterministic to scochatic settings
(2) Set the grid and boundaries for parameters in the stochastic setting
(3) Provide starting points for the optimization

### Running Deterministic Optimization

In [ ]:
import pandas as pd
from scipy.interpolate import interp1d
from Logistics_stoch import CShip
from Logistics_stoch import CRoundtrip
from Logistics_stoch import CJourney
from Logistics_stoch import CPort
import deterministic_opt_func as det_opt

oJourney = CJourney(NrOfRoundtrips = 1,
                    LegsPerRoundtrip = 5,
                    OpprtCostCapitalRate = 0.08,
                    DailyHire_USDperDay = 30000,
                    FutureProfitPotential_USDperDay = 12968)


oShip = CShip(Vmin=10, 
              Vmax=17, 
              DWTscantling=157880, 
              DWTdesign=145900, 
              Lightweight=49000, 
              k=0.00000391, 
              p=381, 
              g=3.1, 
              a=0.666667, 
              ShipDischargeRate=3000, 
              BallastCapacity=54500,
              MinFillRateShip=0.3,
              AuxFuelConsumption_TonnePerDay=5)

oPortList = []

df_data = pd.read_excel('journey_data.xlsx',sheet_name='Ports')

for i in range(0,oJourney.LegsPerRoundtrip+1):
    for index,row in df_data.iterrows():
        v_name = row['Variable']
        v_value = row[i]
        #print("Loading from excel, leg:"+str(i)+", Executing statement("+str(v_name)+"="+str(v_value)+")")
        exec(v_name + '=' + str(v_value))
    
    
    print(CargoRevenueRate_USDperBarrelper1000nm)
    oPort = CPort(PortNr=i, 
                  DistancePreviousPort_nm=DistancePreviousPort_nm,
                  LoadingRate_QbmetresperHr=LoadingRate_QbmetresperHr,
                  WaitingTime_Hrs=WaitingTime_Hrs,              
                  CargoIntake_Barrels=CargoIntake_Barrels,
                  CargoIntake_QBmetresperBarrel=CargoIntake_QBmetresperBarrel,
                  CargoIntake_QbmetresPerTonne=CargoIntake_QbmetresPerTonne,
                  FixedPortAccessCosts_USD = FixedPortAccessCosts_USD,
                  UnloadingCharge_USDperHr = UnloadingCharge_USDperHr,
                  LoadingCharge_USDperHr = LoadingCharge_USDperHr,
                  CargoRevenueRate_USDperBarrelper1000nm = CargoRevenueRate_USDperBarrelper1000nm,
                  MainBunkerRate_USDperBarrel = MainBunkerRate_USDperBarrel,
                  MainBunker_QBmetresperBarrel = MainBunker_QBmetresperBarrel,
                  MainBunker_QbmetresPerTonne = MainBunker_QbmetresPerTonne,
                  AuxFuelRate_USDperTonne = AuxFuelRate_USDperTonne,
                  UnloadingCosts_USD = UnloadingCosts_USD,
                  LoadingCosts_USD = LoadingCosts_USD,
                  CargoIntake_Tonne = CargoIntake_Tonne,
                  UnloadingTime_Hrs = UnloadingTime_Hrs,
                  LoadingTime_Hrs = LoadingTime_Hrs,
                  LegRevenue_Barrels = LegRevenue_Barrels,
                  LegRevenue_USD = LegRevenue_USD)
    
    oPortList.append(oPort)

oRTList_det = det_opt.run_deterministic_opt(oJ=oJourney,
                                            oS=oShip,
                                            oPL=oPortList)
# storing solution in the journey object
oJourney.oRTList_det = oRTList_det


In [ ]:
df_data

# Set up the freight rate dynamics
We assume that the log of the spot freight rate has the Ornstein-Uhlenbeck (OU) dynamics:

$dR_t = \lambda_r (R_t - \bar{R}) dt + \sigma_r dW_t$

The solution of the OU process is well know, the random variable is normally distributed with the following mean and variance:

$\mathbb{E}(R_T|R_0) = R_0 e^{-\lambda_r T} + \bar{R} (1 - e^{-\lambda_r T})$

$Var(R_T|R_0) = \frac{{\sigma_r}^2}{2 \lambda_r}(1 - e^{-2\lambda_r T})$

To get to the actual freight rate:

$\mathbb{E} [exp(R_T)|R_0] = e^{\mathbb{E}(R_T|R_0) + 0.5*Var(R_T|R_0)} = e^{R_0 e^{-\lambda_r T} + \bar{R} (1 - e^{-\lambda_r T}) + \frac{{\sigma_r}^2}{4 \lambda_r}(1 - e^{-2\lambda_r T})}$

$Var[exp(R_T)|R_0] = (e^{Var(R_T|R_0)}-1) e^{2\mathbb{E}(R_T|R_0) + Var(R_T|R_0)} = (e^{\frac{{\sigma_r}^2}{4 \lambda_r}(1 - e^{-2\lambda_r T})}-1) e^{2(R_0 e^{-\lambda_r T} + \bar{R} (1 - e^{-\lambda_r T})) + \frac{{\sigma_r}^2}{4 \lambda_r}(1 - e^{-2\lambda_r T})}$


In [ ]:
# O.U. parameters of the log rate model:
# the data is assuming T=1 it is one year.

import matplotlib.pyplot as plt
import OU_process

ou_process = OU_process.OU_process(r_bar = 6.2, r_lambda=3, r_sigma=0.3)


T_days=15
r = 6.27
states, step = ou_process.gen_states(T_days=T_days)
print('All generated states for T_days=' + str(T_days))
print(states)
p = ou_process.p_vector(states, step, r, T_days=T_days)

plt.xlabel('States')
plt.title('Probability as a function of state in T_days')
plt.plot(states, p)
plt.show()

## Forward Freight Agreements

We assume that the futures curve is trading at a certain discount to the expected spot rate given by the O.U. process. If the spot rate is below its long term average, the expectation curve will look like a contango, and for a spot rate above the long term rate, the expectation curve will look like a backwardation. On both of these curves we apply a discount which is governed by a slope contant (fut_curve_slope in the code):

$F_T = e^{-slope * T} \mathbb{E}(e^{R_T}|R_t)$

In [ ]:
import numpy as np
import FFA
ffa = FFA.FFA(fut_curve_slope = -0.2)

r0 = ou_process.r_bar * 0.9
T_hor = np.array(range(0,365))
futures_curve = []
expecation_curve= []
for t in T_hor:
    E_R = ou_process.E_exp(r0=r0,T_days=t)
    futures_curve.append(ffa.shipping_futures(E_R=E_R,T_days=t))
    expecation_curve.append(E_R)
    
plt.plot(futures_curve,label='Futures')
plt.plot(expecation_curve, label='Model')
plt.plot(np.exp(ou_process.generate_path(r=r0,T_days=365)), label='Simulated Path')
plt.xlabel("Time in days")
plt.ylabel("Price in US$")
plt.title("Futures curve at t=0 vs the model future realization")
if r0<ou_process.r_bar:
    plt.legend(loc="lower right")
else:
    plt.legend(loc="upper right")
plt.show()

## Running the stochastic optimization

In [ ]:
from Optimization_Problem import Optimization_Problem
op0 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, r_lambda=3, r_sigma=0.9), 
                          ffa=FFA.FFA(fut_curve_slope = -0.01), 
                          eta=2e-08) # determines how important is the variance of future spot rates)
oRTList0 = op0.run_optimization()

# Display the results



## Value Function ($G$) or Goodwill 

The final value function after all legs and rountrip were calculated from the back.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter 

plt.plot(oRTList0[oJourney.NrOfRoundtrips].RoundtripGoodwill.x,
         oRTList0[oJourney.NrOfRoundtrips].RoundtripGoodwill.y)
plt.xlabel('State of Freight Market') 
plt.ylabel('Goodwill') 
plt.title('The value of the ship')
plt.gca().yaxis.set_minor_formatter(NullFormatter())
plt.show()



## Hedge Ratio

In [ ]:
plt.plot(oRTList0[1].oLegList[0].Hedge_Ratio.x,
         oRTList0[1].oLegList[0].Hedge_Ratio.y, 
         label='Leg 5')
plt.plot(oRTList0[1].oLegList[1].Hedge_Ratio.x,
         oRTList0[1].oLegList[1].Hedge_Ratio.y, 
         label='Leg 4')
plt.plot(oRTList0[1].oLegList[2].Hedge_Ratio.x,
         oRTList0[1].oLegList[2].Hedge_Ratio.y, 
         label='Leg 3')
plt.plot(oRTList0[1].oLegList[3].Hedge_Ratio.x,
         oRTList0[1].oLegList[3].Hedge_Ratio.y, 
         label='Leg 2')
plt.plot(oRTList0[1].oLegList[4].Hedge_Ratio.x,
         oRTList0[1].oLegList[4].Hedge_Ratio.y, 
         label='Leg 1')
plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='upper right')
plt.title('Optimal hedge ratio for each state')
plt.show()

## Time of trip

In [ ]:
plt.plot(oRTList0[1].oLegList[4].TimeAtSea_Days.x,
         oRTList0[1].oLegList[4].TimeAtSea_Days.y, 
         label='Leg 5')
plt.plot(oRTList0[1].oLegList[3].TimeAtSea_Days.x,
         oRTList0[1].oLegList[3].TimeAtSea_Days.y, 
         label='Leg 4')
plt.plot(oRTList0[1].oLegList[2].TimeAtSea_Days.x,
         oRTList0[1].oLegList[2].TimeAtSea_Days.y, 
         label='Leg 3')
plt.plot(oRTList0[1].oLegList[1].TimeAtSea_Days.x,
         oRTList0[1].oLegList[1].TimeAtSea_Days.y, 
         label='Leg 2')
plt.plot(oRTList0[1].oLegList[0].TimeAtSea_Days.x,
         oRTList0[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 1')
plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='upper right')
plt.title('Time of trip in days')
plt.show()





## Speed trip

In [ ]:
plt.plot(oRTList0[1].oLegList[4].Speed_kn.x,
         oRTList0[1].oLegList[4].Speed_kn.y, 
         label='Leg 5')
plt.plot(oRTList0[1].oLegList[3].Speed_kn.x,
         oRTList0[1].oLegList[3].Speed_kn.y, 
         label='Leg 4')
plt.plot(oRTList0[1].oLegList[2].Speed_kn.x,
         oRTList0[1].oLegList[2].Speed_kn.y, 
         label='Leg 3')
plt.plot(oRTList0[1].oLegList[1].Speed_kn.x,
         oRTList0[1].oLegList[1].Speed_kn.y, 
         label='Leg 2')
plt.plot(oRTList0[1].oLegList[0].Speed_kn.x,
         oRTList0[1].oLegList[0].Speed_kn.y, 
         label='Leg 1')
plt.xlabel('State of Freight Market') 
plt.ylabel('nm/h') 
plt.title('Speeds of the vessel')
plt.show()


## Changing parameters of the problem:

Assume that the forward rates are discounted by more than previously assumed, in this case hadging is more costly on average, we expect less hedging and decrease of the hedge ratio:

> ffa = FFA.FFA(fut_curve_slope = -0.01)

In [ ]:
from Optimization_Problem import Optimization_Problem
op1 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, 
                                                        r_lambda=3, 
                                                        r_sigma=0.3), 
                          ffa=FFA.FFA(fut_curve_slope = -0.02), 
                          eta=0.000000) 
oRTList1 = op1.run_optimization()

## Comparison of hedge ratios

In [ ]:
plt.plot(oRTList0[1].oLegList[0].Hedge_Ratio.x,
         oRTList0[1].oLegList[0].Hedge_Ratio.y, 
         label='Discount=-1% leg=4')
plt.plot(oRTList0[1].oLegList[4].Hedge_Ratio.x,
         oRTList0[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-1% leg=5')

plt.xlabel('State of Freight Market') 
#plt.ylabel('Optimal Hedge Ratio') 
plt.title('Optimal hedge ratio for each state')
plt.legend(loc='lower right')
plt.show()

## Comparison of time of journeys of the last leg

In [ ]:
plt.plot(oRTList0[1].oLegList[0].TimeAtSea_Days.x,
         oRTList0[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-1%')
plt.plot(oRTList1[1].oLegList[0].TimeAtSea_Days.x,
         oRTList1[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-2%')
plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='upper right')
plt.title('Time of trip in days')
plt.show()

## More importance to the volatility of the spot rates 
>eta=0.000001

In [ ]:
op2 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, r_lambda=3, r_sigma=0.3), 
                          ffa=FFA.FFA(fut_curve_slope = -0.02), 
                          eta=0.0000015) 
oRTList2 = op2.run_optimization()

In [ ]:
plt.plot(oRTList1[1].oLegList[4].Hedge_Ratio.x,
         oRTList1[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-1% eta=1e-6')
plt.plot(oRTList2[1].oLegList[4].Hedge_Ratio.x,
         oRTList2[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-2% eta=1e-6')
plt.plot(oRTList1[1].oLegList[4].Hedge_Ratio.x,
         oRTList1[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-2% eta=1.5e-6')

plt.xlabel('State of Freight Market') 
#plt.ylabel('Optimal Hedge Ratio') 
plt.title('Optimal hedge ratio for each state')
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(oRTList0[1].oLegList[0].TimeAtSea_Days.x,
         oRTList0[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-1% eta=1e-6')
plt.plot(oRTList1[1].oLegList[0].TimeAtSea_Days.x,
         oRTList1[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-2% eta=1e-6')
plt.plot(oRTList2[1].oLegList[0].TimeAtSea_Days.x,
         oRTList2[1].oLegList[0].TimeAtSea_Days.y, 
         label='Leg 5 Disc=-2% eta=1.5e-6')

plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='upper right')
plt.title('Time of trip in days')
plt.show()

## Changing paramters of the OU process

> ou_process = OU_process.OU_process(r_bar = 6.2, r_lambda=3, r_sigma=0.3)


In [ ]:
op3 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, r_lambda=0.5, r_sigma=0.3), 
                          ffa=FFA.FFA(fut_curve_slope = -0.05), 
                          eta=0.000001) 
oRTList3 = op3.run_optimization()

op4 = Optimization_Problem(oJourney=oJourney, 
                          oShip=oShip, 
                          oPortList=oPortList, 
                          process=OU_process.OU_process(r_bar = 6.2, r_lambda=0.8, r_sigma=0.3), 
                          ffa=FFA.FFA(fut_curve_slope = -0.05), 
                          eta=0.000001) 
oRTList4 = op4.run_optimization()

In [ ]:
plt.plot(oRTList3[1].oLegList[4].Hedge_Ratio.x,
         oRTList3[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-1% eta=5e-7 r_bar=6.8')
plt.plot(oRTList4[1].oLegList[4].Hedge_Ratio.x,
         oRTList4[1].oLegList[4].Hedge_Ratio.y, 
         label='Discount=-1% eta=5e-7 r_bar=6.2')


plt.xlabel('State of Freight Market') 
#plt.ylabel('Optimal Hedge Ratio') 
plt.title('Optimal hedge ratio for each state')
plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.plot(oRTList3[1].oLegList[4].TimeAtSea_Days.x,
         oRTList3[1].oLegList[4].TimeAtSea_Days.y, 
         label='Leg 5 Discount=-1% eta=1e-7 r_bar=6.2')
plt.plot(oRTList4[1].oLegList[4].TimeAtSea_Days.x,
         oRTList4[1].oLegList[4].TimeAtSea_Days.y, 
         label='Leg 5 Discount=-1% eta=1e-7 r_bar=6.8')

plt.xlabel('State of Freight Market') 
plt.ylabel('Days') 
plt.legend(loc='lower left')
plt.title('Time of trip in days')
plt.show()

## Checking if the function is convex

In [ ]:
ou_process = OU_process.OU_process(r_bar = 6.2, r_lambda=1.8, r_sigma=0.3)
ffa = FFA.FFA(fut_curve_slope = -0.5)

params = [ou_process.r_bar ,oShip, oPortList, oJourney, ou_process, ffa, 0.00002]

In [ ]:
leg = oRTList0[1].oLegList[0]
t_range = np.linspace(leg.TimeAtSeaMin_Days,leg.TimeAtSeaMax_Days,num=100)
h_range = np.linspace(0,1,num=100)
z = np.zeros((100,100))
it = 0
ih = 0
max_z = 0
for t in t_range:
    ih = 0
    for h in h_range:
        z[it,ih],q = leg.CalcGoodwill([h,t],params)
        if z[it,ih]>max_z:
            h_max = h
            t_max = t
            max_z = z[it,ih] 
        ih = ih + 1
    it = it + 1
    

In [ ]:
print([h_max,t_max,max_z])

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
# Read data from a csv
x, y = t_range, h_range
fig = go.Figure(data=[go.Surface(z=z.T, x=x, y=y)])
fig.update_layout(title='Goodwill as a function of time and hedge ratio', autosize=False,
                  width=1000, height=1000,
                  scene = dict(
                    xaxis_title='Time of the leg in days',
                    yaxis_title='Hedge ratio from 0 to 1',
                    zaxis_title='Value of the ship'),
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
leg.FindBestGoodwill(ou_process, ffa, 0, oJourney, oShip, oRTList0, oPortList)

In [ ]:
plt.plot(leg.Hedge_Ratio.x,leg.Hedge_Ratio.y)